In [1]:
import os
import re
import cv2
import numpy as np

In [2]:
X_0 = []
X_1 = []
path = 'liewen'

# X_0_re匹配文件中无裂纹的图片， X_1_re匹配文件中有裂纹的图片
X_0_re = r'\d*\_\d*\.jpg'
X_1_re = r'\d*\_\d*\_3\.jpg'
# 只选取160 * 160 pixel 的文件夹下
match = r'liewen\\lw\d*\\160'

# 将有裂纹的图片名称保存到X_1列表中
# 将无裂纹的图片名称保存到X_0列表中
# 读取path底下所有的160*160 pixel的图片，
for root, dirs, files in os.walk(path):
    if re.match(match, root):
        for fil in files:
            path_name = root + os.sep + fil
            # print(path_name)
            if re.match(X_0_re, fil):
                X_0.append(path_name)
            elif re.match(X_1_re, fil):
                X_1.append(path_name)    

In [3]:
len(X_0)

4291

In [4]:
len(X_1)

114

In [5]:
# 将关键点描述组成输入矩阵X
def get_data(X_0):
    X_fu = []
    for img in X_0:
        # 读取图片
        img = cv2.imread(img, 0)
    
        #  重组矩阵，使其一行表示一张图片，列为 10 * 128
        x = img.reshape((1, -1))
        X_fu.append(x[0])
        
    return np.array(X_fu)

In [6]:
X_fu = get_data(X_0)
X_fu.shape

(4291, 25600)

In [7]:
X_zheng = get_data(X_1)
X = np.vstack((X_fu, X_zheng))
X.shape

(4405, 25600)

In [8]:
y0 = np.array([0] * len(X_fu))
y1 = np.array([1] * len(X_zheng))
# y 为X矩阵中样本的标签， 0表示无裂纹， 1表示有裂纹
y = np.hstack((y0, y1))
y.shape

(4405,)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [24]:
from sklearn import svm

clf = svm.SVC(class_weight='balanced')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
clf.score(X_test, y_test)

0.9712556732223904

In [26]:
y_predict =  clf.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_predict)

array([[1284,    0],
       [  38,    0]], dtype=int64)

In [28]:
from sklearn.metrics import precision_score

precision_score(y_test, y_predict)

F:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

In [29]:
from sklearn.metrics import recall_score

recall_score(y_test, y_predict)

0.0

In [30]:
from sklearn.metrics import f1_score

f1_score(y_test, y_predict)

F:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

In [31]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(class_weight='balanced')
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [19]:
log_reg.score(X_test, y_test)

0.9534619750283768

In [20]:
y_predict = log_reg.predict(X_test)

In [21]:
from sklearn.metrics import precision_score

precision_score(y_test, y_predict)

0.047619047619047616